In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import kaggle

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d mohammadamireshraghi/blood-cell-cancer-all-4class

# Unzip data

In [ ]:
! unzip -q "blood-cell-cancer-all-4class.zip"

# Set dataset path

In [ ]:
datasets_dir = "Blood cell Cancer [ALL]"


# Variable

In [ ]:
epochs = 32
IMG_SIZE = 128
BATCH_SIZE = 32

# Dataset Load

In [ ]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(42)



train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = datasets_dir,
    image_size = (IMG_SIZE, IMG_SIZE),
    label_mode = 'categorical',
    color_mode="rgb",
    batch_size = BATCH_SIZE,
    seed=42,
    shuffle = True,
    validation_split=0.2,
    subset="training"

).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory = datasets_dir,
    image_size = (IMG_SIZE, IMG_SIZE),
    label_mode = 'categorical',
    color_mode="rgb",
    batch_size = BATCH_SIZE,
    seed=42,
    shuffle = True,
    validation_split=0.2,
    subset="validation"

)
class_names = test_dataset.class_names
test_data = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
class_names


# Data augmentation

In [ ]:
import tensorflow as tf

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(IMG_SIZE, IMG_SIZE, 3)),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
  tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
  tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)
], name ="data_augmentation")

# Plot images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10,10))
for image_batch,label_batch in train_data.take(1):
    for i in range(0,8):
        plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(label_batch[i])])
        plt.axis("off")
plt.tight_layout()
plt.show()

# Augmented image view

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in train_data.take(1):
    augmented_image_batch = data_augmentation(image_batch)
    image_plt_len = len(image_batch) if len(image_batch) < 4 else 4
    for i in range(0, image_plt_len):
        plt.subplot(image_plt_len,2,2*i+1)
        plt.imshow(image_batch[i].numpy().astype("uint32"))
        plt.title(class_names[np.argmax(label_batch[i])])
        plt.axis("off")
        plt.subplot(image_plt_len,2,2*i+2)
        img = augmented_image_batch[i].numpy().astype("uint32")
        plt.imshow(img)
        plt.title(f"augmented")
        plt.axis("off")

## Create checkpoint

In [ ]:
# checkpoint_path = "/CheckPoint/"
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     checkpoint_path,
#     # save_weights_only=True,
#     monitor='val_accuracy',
#     save_best_only=True
# )


# Model. transferlearning. ResNet50


In [ ]:
# ResNet 50 model
import keras

model = keras.Sequential()
resnet = keras.applications.ResNet50(
    include_top=False,
    input_shape=(128, 128, 3),
    pooling='avg',
    classes=len(class_names),
    weights='imagenet'
)

for layer in resnet.layers:
  layer.trainable = False

model.add(resnet)
model.add(keras.layers.Flatten())

#model.add(keras.layers.Dense(512, activation= 'relu'))
#model.add(keras.layers.Dropout(rate= 0.2))
model.add(keras.layers.Dense(256, activation= 'relu'))
model.add(keras.layers.Dropout(rate= 0.2))
#model.add(keras.layers.Dense(128, activation= 'relu'))
#model.add(keras.layers.Dropout(rate= 0.2))
#model.add(keras.layers.Dense(128, activation= 'relu'))
#model.add(keras.layers.Dropout(rate= 0.2))
#model.add(keras.layers.Dense(64, activation= 'relu'))
#model.add(keras.layers.Dropout(rate= 0.2))

model.add(keras.layers.Dense(32, activation= 'relu'))

model.add(keras.layers.Dense(len(class_names), activation='softmax', name='output_layer'))


# Model. transferlearning.EfficientNetB0

In [ ]:
# import tensorflow as tf

# # Setup the base model and freeze its layers (this will extract features)
# base_model = tf.keras.applications.EfficientNetB0(include_top=False)
# base_model.trainable= False

# #Setup model architecture with trainable top layers
# inputs = tf.keras.layers.Input(shape=(IMG_SIZE,IMG_SIZE,3), name='input_layer')
# x = data_augmentation(inputs)
# x = base_model(x, training=False)
# x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pooling_layer')(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(64, activation= 'relu')(x)
# x = tf.keras.layers.Dense(64, activation= 'relu')(x)
# x = tf.keras.layers.Dense(16, activation= 'relu')(x)
# outputs = tf.keras.layers.Dense(len(class_names), activation='softmax', name='output_layer')(x)
# model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

# Model ploting

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,        # Set to True to display shapes of the layers
    show_dtype=False,        # Set to True to display data types of the layers
    show_layer_names=True,   # Set to True to display layer names
    rankdir='TB',            # TB for top-bottom direction, LR for left-right direction
    expand_nested=False,     # Set to True to expand nested models
    dpi=96,                  # Dots per inch for the figure resolution
    layer_range=None,        # Range of layers to display
    show_layer_activations=False,  # Set to True to display layer activations
    show_trainable=False     # Set to True to display trainable parameter count
)

# Model compile

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'],
)

# Model train

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2)
]
history = model.fit(
    train_data,
    epochs=32,
    validation_data = test_data,
    validation_steps = len(test_data),
    # callbacks = callbacks
)

# Plot history loss and accuracy

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();


In [ ]:
plot_loss_curves(history)

# Evalute model

In [ ]:
model.evaluate(test_data)

# Best model

In [ ]:
"""clone the model"""
# best_model = tf.keras.models.load_model(checkpoint_path)

# Evalute best model

In [ ]:
# best_model.evaluate(test_data)

# Confusion matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(8, 8), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.tick_params(axis='x', labelrotation=90)
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
y_true = np.array([])
y_pred = np.array([])
for x, y in test_data:
    y_true = np.concatenate([y_true, np.argmax(y, axis=-1)], axis=0)
    y_pred = np.concatenate([y_pred, np.argmax(model.predict(x), axis=-1)], axis=0)

In [ ]:
make_confusion_matrix(y_true, y_pred, class_names)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(6, 4))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in np.ndindex(cm.shape):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\n Accuracy={:0.4f}; Misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

# Example usage:
matrix = [[69, 0, 3, 14],
          [3, 196, 0, 0],
          [1, 0, 155, 1],
          [4, 2, 0, 200]]

plot_confusion_matrix(cm=np.array(matrix),
                      normalize=False,
                      target_names=['EarlyPreB', 'PreB', 'ProB', 'benign'],
                      title="Confusion Matrix")

plot_confusion_matrix(cm=np.array(matrix),
                      normalize=True,
                      target_names=['EarlyPreB', 'PreB', 'ProB', 'benign'],
                      title="Confusion Matrix, Normalized")


In [ ]:
y_pred


In [ ]:
import matplotlib.pyplot as plt

# Initialize counters for correct and incorrect predictions
correct_count = 0
incorrect_count = 0

plt.figure(figsize=(10, 10))
for image_batch, label_batch in test_data.take(1):
    for i in range(8):  # Show 8 images
        plt.subplot(3, 4, i + 1)
        img = image_batch[i].numpy().astype("uint8")
        plt.imshow(img)
        true_class = class_names[np.argmax(label_batch[i])]

        # Expand the image dimensions and make a prediction
        img = np.expand_dims(img, axis=0)
        predict_class = model.predict(img)[0]
        predict_class = class_names[np.argmax(predict_class)]

        title = f"True: {true_class}\nPredicted: {predict_class}"
        if true_class == predict_class:
            plt.title(title, color='g')  # Correct prediction shown in green
            correct_count += 1
        else:
            plt.title(title, color='r')  # Incorrect prediction shown in red
            incorrect_count += 1
        plt.axis("off")

plt.tight_layout()
plt.show()